In [ ]:
"""
{"version": "v2.0", 
 "data": [{"title": "Normans", 
    "paragraphs": 
    [{"qas": 
        [ 
         {"question": "When were the Normans in Normandy?", 
          "id": "56ddde6b9a695914005b9629", 
          "answers": [{"text": "10th and 11th centuries", "answer_start": 94}, 
                      {"text": "in the 10th and 11th centuries", "answer_start": 87}, 
                      {"text": "10th and 11th centuries", "answer_start": 94}, 
                      {"text": "10th and 11th centuries", "answer_start": 94}], 
          "is_impossible": false}, 
         {"question": "What century did the Normans first gain their separate identity?", 
          "id": "56ddde6b9a695914005b962c", 
          "answers": [{"text": "10th century", "answer_start": 671}, 
                      {"text": "the first half of the 10th century", "answer_start": 649}, 
                      {"text": "10th", "answer_start": 671}, 
                      {"text": "10th", "answer_start": 671}], 
          "is_impossible": false}, 
         {"plausible_answers": [{"text": "Normandy", "answer_start": 137}], 
          "question": "What is France a region of?", 
          "id": "5ad39d53604f3c001a3fe8d2", 
          "answers": [], 
          "is_impossible": true}, 
         {"plausible_answers": [{"text": "Rollo", "answer_start": 308}], 
          "question": "Who did King Charles III swear fealty to?", 
          "id": "5ad39d53604f3c001a3fe8d3", 
          "answers": [], 
          "is_impossible": true}], 
        "context": "The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries."
     }
        ]}]}
"""

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')
def get_sentence(context, start):
    """Get sentence include start
    """
    doc = nlp(context)
    cur = 0
    for sent in doc.sents:
        if len(sent.text) + cur > start:
            return sent.text
        cur += len(sent.text)
    return ''
    
text = u"this is a sentence...hello...and another sentence."
print(f'sentence including `hello` is` {get_sentence(text, 21)}`')

sentence including `hello` is` hello...and another sentence.`


In [12]:
from collections import Counter
def vote(answers):
    """Choose the answer in answers where answer.text appeare most of time
    """
    ct = Counter([answer['text'] for answer in answers])
    choose = answers[0]
    max_count = 1
    for answer in answers:
        if ct[answer['text']] > max_count:
            max_count = ct[answer['text']]
            choose = answer
    return choose

print(vote([{'text':'aaa'},{'text':'bbb'},{'text':'bbb'}]))

{'text': 'bbb'}


In [19]:
import ujson as json

def get_data(file_path):
    """Read data from file_path, convert data into form
        {
            'context': str
            'gold_sentence': str
            'question': str
            'label': yes/no yes means the question has answer from the context, 
                     or no means there is no answer
        }
    """
    with open(file_path) as fh:
        squad_dev = json.load(fh)['data']
    all_data = []
    without_plausible = 0
    for title in squad_dev:
        for paragraph in title['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                # get {'text':xxx, 'answer_start':xxx}
                answer = {}
                if qa['is_impossible']:
                    if qa['plausible_answers']:
                        answer = qa['plausible_answers'][0]
                else:
                    answer = vote(qa['answers'])
                gold_sentence = get_sentence(context, answer['answer_start']) if answer else ''
                question = qa['question']
                label = 0 if qa['is_impossible'] else 1
                all_data.append({
                    'context': context,
                    'gold_sentence': gold_sentence,
                    'question': question,
                    'label': label,
                    'answer': answer
                })
    return all_data
all_data = get_data('./external/dev-v2.0.json')
with open('./external/dev-v2.0-preprocessed.json', 'w') as fh:
    json.dump(all_data, fh)
print(f'total_num for data is {len(all_data)}')
print(f'first 10 data is {all_data[:10]}')

total_num for data is 11873
first 10 data is [{'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'gold_sentence': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France.', 'question